# GitHub Pull Requests: EDA Notebook

This notebook shows how to:
1. Load raw GitHub PR JSON data.
2. Flatten the nested data structure into a Pandas DataFrame.
3. Subset columns to keep only those we care about.
4. Perform basic exploratory data analysis (EDA).

---

## 1. Setup & Imports

I begin by importing our required libraries:

In [17]:
import pandas as pd
import json

## 2. Load & Flatten the PR JSON Data

I already have a file named `github_prs_raw.json` inside the `../data/` folder. Let's open it, parse the JSON, and create a DataFrame using `pd.json_normalize`.


In [ ]:
# Open the raw PR data from a JSON file
with open("../data/github_prs_raw.json", "r") as f:
    raw_pr_data = json.load(f)

# Flatten the nested JSON structure
df_all = pd.json_normalize(raw_pr_data)

# Let's see how many columns we have and show a preview
print(f"Total columns: {df_all.shape[1]}")
df_all.head()  # Just show the head
df_all.shape


Total columns: 305


TypeError: 'tuple' object is not callable

## 3. Inspect All Column Names

I'll list out all the flattened columns to decide which ones are relevant for my analysis.

In [ ]:
all_columns = df_all.columns.tolist()
print(all_columns)

### Observing the Columns

Here, I can see columns like `number`, `state`, `title`, `body`, `merged_at`, etc.  
For a simpler EDA, let's select a small set of columns.

---

## 4. Select Key Columns

I'll define a list of columns that are present and relevant (based on our earlier inspection). I then create a smaller DataFrame `df` with just those columns.


In [ ]:
# Pick columns you actually have in df_all
selected_columns = [
    "number",        # PR number
    "state",         # open/closed
    "title",         # PR title
    "body",          # PR description
    "created_at",    # Creation timestamp
    "updated_at",    # Last updated timestamp
    "closed_at",     # If closed
    "merged_at",     # If merged
    "assignee",      # Single assignee (nullable)
    "assignees",     # List of assigned users
    "labels",        # Label objects
    "user.login"     # Author's username
]

# Ensure we only include columns that exist
existing_cols = [col for col in selected_columns if col in df_all.columns]

df = df_all[existing_cols].copy()

print(f"Columns in df: {df.columns.tolist()}")
df.head()


## 6. Visualize Missing Data

Certain columns (like `assignee` or `assignees`) might be empty or `None` across most rows. Let's plot a heatmap of missing values for a subset of the columns (e.g., the first 50) to see the overall pattern.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming your DataFrame is named df
plt.figure(figsize=(12, 8))

# We'll just take the first 50 columns to avoid crowding
subset = df.iloc[:, :50]

sns.heatmap(subset.isnull(), cbar=False, cmap="viridis")
plt.title("Missing Values Heatmap (First 50 Columns)")
plt.xlabel("Columns")
plt.ylabel("Rows")
plt.show()


## 7. Drop Columns with Excessive Missing Values

From the heatmap, I see `assignee` is nearly always `None`. Let's remove it (and any other columns that meet a certain missing threshold).

In [ ]:
# Decide on a threshold; e.g., drop columns missing in over 90% of rows
threshold = 0.8

# Calculate the percentage of NaN in each column
missing_percent = df.isnull().mean()

# Filter columns that exceed our threshold
cols_to_drop = missing_percent[missing_percent > threshold].index
print("Dropping columns:", cols_to_drop.tolist())

# Drop them
df.drop(columns=cols_to_drop, inplace=True)

# Confirm they’re gone
df.head()


In [ ]:
import numpy as np

def empty_list_to_nan(x):
    # If x is a list and it's empty, return NaN;
    # otherwise return x as-is.
    if isinstance(x, list) and len(x) == 0:
        return np.nan
    return x

# Drop the 'assignees' column entirely
df.drop(columns="assignees", inplace=True, errors="ignore")

# Double-check that 'assignees' is gone
print("Columns after drop:", df.columns.tolist())


In [ ]:
df.head()

## 8. Drop Missing Times

If a pull request is never merged, or if `merged_at` is null, I can't calculate a valid time-to-merge. Let's remove rows where the merge time is missing.


In [ ]:
# Convert created_at / merged_at to datetime if not already done
df["created_dt"] = pd.to_datetime(df["created_at"], errors="coerce")
df["merged_dt"] = pd.to_datetime(df["merged_at"], errors="coerce")

# Compute time_to_merge_hours
df["time_to_merge_hours"] = (df["merged_dt"] - df["created_dt"]).dt.total_seconds() / 3600

# Some PRs might never merge; let's drop rows missing this
before_drop = len(df)
df.dropna(subset=["time_to_merge_hours"], inplace=True)
after_drop = len(df)

print(f"Dropped {before_drop - after_drop} rows where 'time_to_merge_hours' was NaN.")
df[["number", "state", "time_to_merge_hours"]].head(100)

In [ ]:
# Assume df_final is your cleaned and processed DataFrame in your EDA notebook
df[["number", "title", "time_to_merge_hours"]].to_csv("/Users/cememirsenyurt/github-pr-time-estimator/ml/data/processed_pr_data.csv", index=False)
print("Processed data saved to ../data/processed_pr_data.csv")